In [41]:
import cv2
import numpy as np
import math 

In [34]:
#hand = cv2.imread('/home/piyush/Downloads/hand.jpg',0)

In [35]:
#ret, the = cv2.threshold(hand,70,255,cv2.THRESH_BINARY)

In [36]:
#_,contours,_ = cv2.findContours(the.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [37]:
#hull = [cv2.convexHull(c) for c in contours]

In [38]:
#final = cv2.drawContours(hand,hull,-1,(255,0,0))

In [40]:
#cv2.imshow('Orignal Image',hand)
#cv2.imshow('Threshold',the)
#cv2.imshow('Convex Hull',final)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [55]:
capture = cv2.VideoCapture(0)
while(True):
    
    #Capture frames from capture
    ret,frame = capture.read()
    
    #Get hannd from rectangle sub window
    cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0)
    crop_image = frame[100:300,100:300]
    
    #Apply gaussian blur
    blur = cv2.GaussianBlur(crop_image,(3,3),0)
    
    #Change color from Bgr to Hsv
    hsv = cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)
    
    #mask image to make skin color to white and rest to black
    mask = cv2.inRange(hsv , np.array([2,0,0]), np.array([20,255,255]))
    
    #kernel for morphological transformation
    kernel = np.ones((5,5))
    
    #erosion and dilation to remove background noises
    erosion = cv2.erode(mask, kernel, iterations=1)
    dilation = cv2.dilate(erosion, kernel, iterations=1)
    
    #applying gausian blur and threshold
    filtered = cv2.GaussianBlur(dilation, (3,3),0)
    ret, the = cv2.threshold(filtered, 127, 255, 0)
    
    #Show Threshold Image
    cv2.imshow('Thresholded',the)
    
    #Find Contours
    image, contours, heirachy = cv2.findContours(the.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    try:
        #Find Contours with maximum area
        contour = max(contours, key=lambda x:cv2.contourArea(x))
        
        #Create bounding rectangle around contour
        x,y,w,h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image,(x,y),(x+w,y+h),(0,0,255),0)
        
        #Find convexHull
        hull = cv2.convexHull(contour)
        
        #Draw Contours
        drawing = np.zeros(crop_image.shape, np.uint8)
        cv2.drawContours(drawing, [contour], -1, (255,0,0), 0)
        cv2.drawContours(drawing, [hull], -1, (255,0,0), 0)
        
        #Find Convexity defects
        hull = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour, hull)
        
        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

            # if angle > 90 draw a circle at the far point
            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_image, far, 1, [0, 0, 255], -1)

            cv2.line(crop_image, start, end, [0, 255, 0], 2)

        # Print number of fingers
        if count_defects == 0:
            cv2.putText(frame, "ONE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
        elif count_defects == 1:
            cv2.putText(frame, "TWO", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 2:
            cv2.putText(frame, "THREE", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 3:
            cv2.putText(frame, "FOUR", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 4:
            cv2.putText(frame, "FIVE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        else:
            pass
    except:
        pass

    # Show required images
    cv2.imshow("Gesture", frame)
    all_image = np.hstack((drawing, crop_image))
    cv2.imshow('Contours', all_image)

    # Close the camera if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
capture.release()
cv2.destroyAllWindows()

In [52]:
capture.release()